# Monitor en tiempo real USD/DOP

Este cuaderno permite inspeccionar los datos almacenados por `cambio-dollar`, generar un consenso actualizado entre proveedores y visualizar el historial con gráficas interactivas.

Ejecuta primero las celdas superiores y luego usa `monitor_market` para refrescar automáticamente.

In [ ]:
import sqlite3
from datetime import datetime
from pathlib import Path
import time

import pandas as pd
import plotly.express as px
from IPython.display import display

from cambio_dollar.config import get_settings
from cambio_dollar.data_provider import MarketDataService
from cambio_dollar.repository import MarketRepository

In [ ]:
settings = get_settings()
DB_PATH = Path(settings.db_path).resolve()
repository = MarketRepository(DB_PATH)

def load_rate_history(limit: int = 300) -> pd.DataFrame:
    conn = sqlite3.connect(DB_PATH)
    try:
        df = pd.read_sql_query(
            """
            SELECT timestamp, buy_rate, sell_rate, source
            FROM rate_snapshots
            ORDER BY timestamp DESC
            LIMIT ?
            """,
            conn,
            params=(limit,),
        )
    finally:
        conn.close()
    if df.empty:
        return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df.sort_values("timestamp").reset_index(drop=True)

def latest_consensus():
    service = MarketDataService(repository, settings)
    try:
        return service.consensus_from_repository()
    except RuntimeError:
        return None
    finally:
        service.close()

def consensus_to_dataframe(consensus) -> pd.DataFrame:
    if consensus is None:
        return pd.DataFrame()
    summary = {
        "timestamp": [consensus.timestamp],
        "compra": [consensus.buy_rate],
        "venta": [consensus.sell_rate],
        "divergencia": [consensus.divergence_range],
        "proveedores": [" | \ 